In [31]:
from glob import glob
from tqdm import tqdm
import numpy as np

In [3]:
def load_poses(file_path):
    poses = []
    with open(file_path, 'r') as f:
        for line in f:
            pose = np.fromstring(line.strip(), sep=' ').reshape(3, 4)
            poses.append(pose)
    return np.array(poses)

# Example: Load poses.txt
file_path = "./../daeyoung/SemanticKITTI/sequences/07/poses.txt"  # 파일 경로를 입력하세요
poses = load_poses(file_path)

# 첫 번째 프레임의 4x4 변환 행렬 출력
print("First frame pose matrix:")
print(poses[0])

First frame pose matrix:
[[ 1.00000e+00  0.00000e+00  1.67347e-10  0.00000e+00]
 [ 9.31323e-10  1.00000e+00  0.00000e+00  0.00000e+00]
 [-4.14730e-10  0.00000e+00  1.00000e+00  0.00000e+00]]


In [ ]:
def load_point_cloud(file_path):
    point_cloud = np.fromfile(file_path, dtype=np.float32).reshape(-1, 4)
    return point_cloud[:, :3]  # XYZ 값만 사용

point_cloud_path = './../daeyoung/SemanticKITTI/sequences/07/velodyne/*.bin'
point_cloud_list = glob('./../daeyoung/SemanticKITTI/sequences/07/velodyne/*.bin')

point_cloud = load_point_cloud(point_cloud_list[0])

In [12]:
def extend_pose(pose):
    pose_extended = np.eye(4)
    pose_extended[:3, :] = pose
    return pose_extended

extended_poses = [extend_pose(p) for p in poses]

In [15]:
def transform_point_cloud(point_cloud, pose):
    num_points = point_cloud.shape[0]
    points_homo = np.hstack((point_cloud, np.ones((num_points, 1))))  # 동차 좌표로 확장
    transformed_points = (pose @ points_homo.T).T
    return transformed_points[:, :3]  # XYZ 좌표만 반환

transformed_point_cloud = transform_point_cloud(point_cloud, extended_poses[0])

In [21]:
transformed_point_cloud.shape

(126278, 3)

In [32]:
global_point_cloud = []

for i, pose in tqdm(enumerate(extended_poses)):
    point_cloud_file = f"./../daeyoung/SemanticKITTI/sequences/07/velodyne/{i:06d}.bin"
    point_cloud = load_point_cloud(point_cloud_file)
    transformed_point_cloud = transform_point_cloud(point_cloud, pose)
    global_point_cloud.append(transformed_point_cloud)

global_point_cloud = np.vstack(global_point_cloud)


0it [00:00, ?it/s]

1101it [00:03, 287.55it/s]


In [34]:
import open3d as o3d

def visualize_point_cloud(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    o3d.visualization.draw_geometries([pcd])

def save_point_cloud(points, save_path):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    o3d.io.write_point_cloud(save_path, pcd)

save_point_cloud(global_point_cloud, './../data/SemanticKITTI/global_point_cloud.pcd')
# visualize_point_cloud(global_point_cloud)

In [35]:
global_point_cloud.shape

(133585213, 3)